In [32]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [53]:
df = pd.read_csv('https://opendata-ajuntament.barcelona.cat/resources/bcn/Asia_Allotjaments/B002_Hotels.csv')
print(df.shape)
df.head()

(884, 33)


,CODI_EQUIPAMENT,EQUIPAMENT,SECCIO,TIPUS_VIA,NOM_CARRER,NUM_CARRER_1,NUM_CARRER_2,CODI_BARRI,NUM_BARRI,CODI_DISTRICTE,NOM_DISTRICTE,CODI_POSTAL,CODI_POBLACIO,POBLACIO,LATITUD,LONGITUD,X_ETRS89,Y_ETRS89,X_ED50,Y_ED50,TELEFON_NUM,TELEFON_TIPUS,TELEFON_INFO_COM,HORARI_PERIODE_INICI,HORARI_PERIODE_FI,HORARI_DIES,HORARI_HORES_INICI,HORARI_HORES_FI,HORARI_OBSERVACIONS,3ER_NIVELL,2N_NIVELL,1ER_NIVELL,Unnamed: 32
0,12124527,Hotel Amrey Sant Pau - HB-004046,#,C,Sant Antoni Maria Claret,173,173.0,35,el Guinardó,7,Horta-Guinardó,8041,19,BARCELONA,41.414013,2.177888,431293.278,4585044.210,431385.218,4585245.172,934335151,Tel.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hotels 2 estr.,Hotels,Allotjament,NaN
1,12124527,Hotel Amrey Sant Pau - HB-004046,#,C,Sant Antoni Maria Claret,173,173.0,35,el Guinardó,7,Horta-Guinardó,8041,19,BARCELONA,41.414013,2.177888,431293.278,4585044.210,431385.218,4585245.172,934335151,Tel.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 estrelles,Hotels,Allotjament,NaN
2,75145812,Hotel Sansi Pedralbes - HB-004086,#,Av,Pearson,1,3.0,21,Pedralbes,4,Les Corts,8034,19,BARCELONA,41.393699,2.111575,425728.073,4582843.677,425819.962,4583044.618,932063880,Centraleta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hotels 4 estr.,Hotels,Allotjament,NaN
3,75145812,Hotel Sansi Pedralbes - HB-004086,#,Av,Pearson,1,3.0,21,Pedralbes,4,Les Corts,8034,19,BARCELONA,41.393699,2.111575,425728.073,4582843.677,425819.962,4583044.618,932063880,Centraleta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 estrelles,Hotels,Allotjament,NaN
4,139120246,Hotel Silken Sant Gervasi - HB-004054,#,C,Sant Gervasi de Cassoles,26,26.0,25,Sant Gervasi - la Bonanova,5,Sarrià-Sant Gervasi,8022,19,BARCELONA,41.404710,2.138854,428020.768,4584043.014,428112.678,4584243.966,932531740,Centraleta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 estrelles,Hotels,Allotjament,NaN


In [81]:
address = 'Barcelona, Barcelona'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Barcelona are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinates of Barcelona are 41.3828939, 2.1774322.


In [93]:
df_bcn = df[["NUM_BARRI","LATITUD", "LONGITUD"]]
df_bcn.head(20)

,NUM_BARRI,LATITUD,LONGITUD
0,el Guinardó,41.414013,2.177888
1,el Guinardó,41.414013,2.177888
2,Pedralbes,41.393699,2.111575
3,Pedralbes,41.393699,2.111575
4,Sant Gervasi - la Bonanova,41.404710,2.138854
5,Sant Gervasi - la Bonanova,41.404710,2.138854
6,el Guinardó,41.405424,2.212221
7,el Guinardó,41.405424,2.212221
8,Sants,41.382599,2.140045
9,Sants,41.382599,2.140045


In [89]:
coordinates_df = pd.DataFrame( data = {

'NUM_BARRI' : ['el Guinardó', 'Poblenou', 'Retiro', 'Salamanca', 'Chamartin',
                'Tetuán', 'Chamberí', 'Fuencarral-El Pardo', 'Moncloa-Aravaca', 'Latina', 'Carabanchel',
                'Usera', 'Puente de Vallecas', 'Moratalaz', 'Ciudad Lineal', 'Hortaleza', 'Villaverde',
                'Villa de Vallecas', 'Vicálvaro', 'San Blas-Canillejas', 'Barajas'],
'LATITUD' : ['41.405424	', '2.212221', '40.408072', '40.43', '40.453333', '40.460556', '40.432792', '40.478611', '40.435151',
                 '40.402461', '40.383669', '40.381336', '40.398204', '40.409869', '40.45', '40.469457', '40.345925', '40.3796', 
                 '40.4042', '40.426001', '40.470196'],
'LONGITUD' : ['-3.707371', '-3.695403', '-3.676729', '-3.677778', '-3.6775', '-3.7', '-3.697186', '-3.709722', '-3.718765', 
                  '-3.741294', '-3.727989', '-3.706856', '-3.669059', '-3.644436', '-3.65', '-3.640482', '-3.709356', '-3.62135', 
                  '-3.60806', '-3.612764', '-3.58489']})

In [83]:
coordinates_df.head()

,Num_Barri,Latitud,Longitud
0,Pedralbes,40.415347,-3.707371
1,Poblenou,40.402733,-3.695403
2,Retiro,40.408072,-3.676729
3,Salamanca,40.43,-3.677778
4,Chamartin,40.453333,-3.6775


In [71]:
coordinates_df.info

<bound method DataFrame.info of            Neighborhood   Latitude  Longitude
0                Centro  40.415347  -3.707371
1            Arganzuela  40.402733  -3.695403
2                Retiro  40.408072  -3.676729
3             Salamanca      40.43  -3.677778
4             Chamartin  40.453333    -3.6775
5                Tetuán  40.460556       -3.7
6              Chamberí  40.432792  -3.697186
7   Fuencarral-El Pardo  40.478611  -3.709722
8       Moncloa-Aravaca  40.435151  -3.718765
9                Latina  40.402461  -3.741294
10          Carabanchel  40.383669  -3.727989
11                Usera  40.381336  -3.706856
12   Puente de Vallecas  40.398204  -3.669059
13            Moratalaz  40.409869  -3.644436
14        Ciudad Lineal      40.45      -3.65
15            Hortaleza  40.469457  -3.640482
16           Villaverde  40.345925  -3.709356
17    Villa de Vallecas    40.3796   -3.62135
18            Vicálvaro    40.4042   -3.60806
19  San Blas-Canillejas  40.426001  -3.612764
20

In [74]:
Madrid_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(coordinates_df['Latitude'], coordinates_df['Longitude'], coordinates_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(Madrid_map)  
    
Madrid_map

In [55]:
CLIENT_ID = 'WCGBIRYHM4CV1PM2MJXISE2ROWZSYBOI1FOZGADY3TXOXX3L' # your Foursquare ID
CLIENT_SECRET = '1U2JLJTZNPU0A1ENYN5VCK1JM05SY0WCSMSR1CBXAWYJQY0X' # your Foursquare Secret
VERSION = '20200722' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WCGBIRYHM4CV1PM2MJXISE2ROWZSYBOI1FOZGADY3TXOXX3L
CLIENT_SECRET:1U2JLJTZNPU0A1ENYN5VCK1JM05SY0WCSMSR1CBXAWYJQY0X


In [75]:
coordinates_df.loc[0, 'Neighborhood']

'Centro'

In [76]:
neighborhood_latitude = coordinates_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = coordinates_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = coordinates_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Centro are 40.415347, -3.707371.


In [77]:
LIMIT= 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [78]:

results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '5f186b52f753817df201c56c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Plaza Mayor',
  'headerFullLocation': 'Plaza Mayor, Madrid',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 98,
  'suggestedBounds': {'ne': {'lat': 40.4198470045, 'lng': -3.701471589889844},
   'sw': {'lat': 40.410846995499995, 'lng': -3.7132704101101566}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4adcda37f964a520193c21e3',
       'name': 'Plaza Mayor',
       'location': {'address': 'Pl. Mayor',
        'lat': 40.415526598647354,
        'lng': -3.7075061894121037,
        'distance': 23,
        'postalCode': '28012',
        'cc': 'ES',
      

In [80]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Plaza Mayor,Plaza,40.415527,-3.707506
1,The Hat Madrid,Hotel,40.414343,-3.707120
2,La Taberna de Mister Pinkleton,Other Nightlife,40.414536,-3.708108
3,Mercado de San Miguel,Market,40.415443,-3.708943
4,Plaza Santa Cruz,Plaza,40.415063,-3.705661
